In [34]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem.QED import qed
import joblib
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizerFast
import torch
import pandas as pd
from tqdm.auto import tqdm

from src.llm_interface import PROMPT, get_answer
from src.utils import all_rools_valid

In [2]:
model_name_or_id = "OpenDFM/ChemDFM-v1.5-8B"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_id,
    torch_dtype=torch.float16,
    device_map="balanced",
    max_memory={0: '10GB', 1: '10GB'}
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.58s/it]


In [35]:
estimation_model, feature_names = list(joblib.load('models/model_iter_1').values())

desc_dict = dict(Descriptors.descList)

descriptor_funcs = []
for f_name in feature_names:
    descriptor_funcs.append((f_name, desc_dict[f_name]))

In [55]:
initial_molecules = \
pd.read_csv('data/quantitive_neftekod25_data.csv')['SMILES'].drop_duplicates().tolist() +\
pd.read_csv('data/export.csv')['SMILES'].tolist()

In [56]:
initial_molecules = [s for s in initial_molecules if all_rools_valid(s)]

In [57]:
initial_molecules

['C1=CC=C(C=C1)NC2=CC=CC=C2',
 'CC(C)(C)CC(C)(C)C1=CC=CC=C1NC2=CC=CC3=CC=CC=C32',
 'C1(=CC=CC=C1N(C2=CC=CC=C2CCCCCCCCC)[H])CCCCCCCCC',
 'C1=CC=C(C=C1)NC2=CC=CC3=CC=CC=C32',
 'CC1=C(C(=CC=C1)O)C',
 'CC1=CC(=C(C=C1)C)O',
 'CC1=C(C=C(C=C1)O)C',
 'CC1=CC(=CC(=C1)O)C',
 'CCC1=CC=C(C=C1)O',
 'CC1=CC(=C(C(=C1)C(C)(C)C)O)C(C)(C)C',
 'CC1=CC(=C(C(=C1)C(C)(C)C)O)CC2=C(C(=CC(=C2)C)C(C)(C)C)O',
 'CC(C)(C)C1=CC(=CC(=C1O)C(C)(C)C)CC2=CC(=C(C(=C2)C(C)(C)C)O)C(C)(C)C',
 'C1(=C(C=C(C=C1C(C)(C)C)CCC(=O)OC)C(C)(C)C)O[H]',
 'Cc1cc(O)c(C(C)(C)C)c(C)c1O',
 'Cc1cc(C(C)(C)C)c(O)c(C(C)(C)C)c1N',
 'CSc1cc(C(C)(C)C)c(N)c(C(C)(C)C)c1',
 'CSc1cc(C(C)(C)C)c(O)c(C(C)(C)C)c1N',
 'CNC(=O)c1cc(C(C)(C)C)c(N)c(C(C)(C)C)c1',
 'CNC(=O)c1cc(C(C)(C)C)c(O)c(C(C)(C)C)c1N',
 'CCOC(=O)c1cc(C(C)(C)C)c(N)c(C(C)(C)C)c1',
 'CCOC(=O)c1cc(C(C)(C)C)c(O)c(C(C)(C)C)c1N',
 'CCOC(=O)c1cc(C(C)(C)C)c(N)c(C(C)(C)C)c1O',
 'CC1=CC(=C(C(=C1)C(C)(C)C)O)CC2=C(C(=CC(=C2)C)C(C)(C)C)OCC1=CC(=C(C(=C1)C(C)(C)C)O)CC2=C(C(=CC(=C2)C)C(C)(C)C)O',
 'Nc1cc(-

In [58]:
class GeneticAlgorithm:
    def __init__(self, mutation_func, crossover_func, fitness_func, population_size, mutation_prob, crossover_prob):

        self.mutation_func = mutation_func
        self.crossover_func = crossover_func
        self.fitness_func = fitness_func
        self.population_size = population_size
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        self.log = []

    def run(self, initial_population, iterations):

        population = initial_population.copy()

        for _ in range(iterations):

            fitness_values = [self.fitness_func(ind) for ind in population]
            
            min_fitness = min(fitness_values)
            max_fitness = max(fitness_values)
            avg_fitness = sum(fitness_values) / len(fitness_values)
            sorted_fitness = sorted(fitness_values)
            median_fitness = sorted_fitness[len(sorted_fitness) // 2] if len(sorted_fitness) % 2 == 1 else (
                sorted_fitness[len(sorted_fitness) // 2 - 1] + sorted_fitness[len(sorted_fitness) // 2])/ 2
            
            self.log.append({
                'min': min_fitness,
                'max': max_fitness,
                'avg': avg_fitness,
                'median': median_fitness
            })

            print(self.log[-1])
            pd.DataFrame({'SMILES': population, 'pred_pdsc': fitness_values}).to_csv('gen_algo_cache/population.csv', index=False)
            
            offspring = []
            for i in tqdm(range(min(len(population), self.population_size))):

                if random.random() < self.crossover_prob:
                    parent1 = self._tournament_selection(population, fitness_values)
                    parent2 = self._tournament_selection(population, fitness_values)
                    child1, child2 = self.crossover_func(parent1, parent2)
    
                    offspring.extend([child1, child2])
                

                if random.random() < self.mutation_prob:
                    mut_child = self.mutation_func(population[i])
                    offspring.append(mut_child)

            combined = population + offspring
            combined = list(set(combined))
            combined.sort(key=lambda x: -self.fitness_func(x))
            population = combined[:self.population_size]
        
        return population, self.log

    def _tournament_selection(self, population, fitness_values, tournament_size=3):
        participants = random.sample(list(zip(population, fitness_values)), tournament_size)
        participants.sort(key=lambda x: -x[1])
        return participants[0][0]

In [59]:
def calculate_fitness(smiles: str) -> float:
    
    mol = Chem.MolFromSmiles(smiles)
    descriptors = {desc_name: desc_f(mol) for desc_name, desc_f in descriptor_funcs}
    X = pd.DataFrame([descriptors])
    prediction = estimation_model.predict(X)[0]
    
    return prediction

def mutation_llm(smiles: str, max_attempts: int=10) -> str:

    for i in range(max_attempts):
        try:
            new_smiles = get_answer(prompt=PROMPT, n_molecules=1, example_smiles=smiles, model=model)[0]

            assert all_rools_valid(new_smiles)

            return new_smiles

            
        except Exception as e:
            continue
            
    return smiles

def crossover_llm(smiles1: str, smiles2: str, max_attempts=10):
    
    for i in range(max_attempts):
        try:
            
            new_smiles1 = get_answer(prompt=PROMPT, n_molecules=1, example_smiles=[smiles1, smiles2], model=model)[0]
            new_smiles2 = get_answer(prompt=PROMPT, n_molecules=1, example_smiles=[smiles1, smiles2], model=model)[0]
            
            if all_rools_valid(new_smiles1) and all_rools_valid(new_smiles2):
                return new_smiles1, new_smiles2
            
        except Exception as e:
            continue
            
    return smiles1, smiles2

In [60]:
llm_genetic_algo = GeneticAlgorithm(
    mutation_llm,
    crossover_llm,
    calculate_fitness,
    population_size=100,
    mutation_prob=0.8,
    crossover_prob=0.2
)

llm_genetic_algo.run(initial_molecules, 10)

{'min': 0.34483998527576176, 'max': 226.32312679309524, 'avg': 54.497945873930995, 'median': 14.04786935417129}


 74%|███████▍  | 32/43 [04:17<04:50, 26.36s/it][18:16:29] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 15 16
[18:16:31] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 15 16
[18:16:36] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 15 16
[18:16:40] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 15 16
[18:16:42] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 16 17 18 19 20 21
100%|██████████| 43/43 [05:49<00:00,  8.13s/it]


{'min': -4.20219574133273, 'max': 226.35277583914765, 'avg': 55.878034596109146, 'median': 19.819478662020536}


 17%|█▋        | 12/71 [01:14<06:16,  6.38s/it][18:19:16] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 8 9 16 24
[18:19:18] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 9 10
[18:19:21] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 7 9 10
 83%|████████▎ | 59/71 [06:41<00:53,  4.48s/it][18:24:47] SMILES Parse Error: syntax error while parsing: Hereisauniqueantioxidantsupplementmoleculethatmeetsallthegivencriteria:
[18:24:47] SMILES Parse Error: check for mistakes around position 1:
[18:24:47] Hereisauniqueantioxidantsupplementmolecul
[18:24:47] ^
[18:24:47] SMILES Parse Error: Failed parsing SMILES 'Hereisauniqueantioxidantsupplementmoleculethatmeetsallthegivencriteria:' for input: 'Hereisauniqueantioxidantsupplementmoleculethatmeetsallthegivencriteria:'
100%|██████████| 71/71 [07:40<00:00,  6.48s/it]


{'min': 9.230725270546763, 'max': 226.35277583914765, 'avg': 77.85233991510022, 'median': 59.77228960756985}


...

{'min': 100.91741392519373, 'max': 233.04558899989934, 'avg': 154.20434616204335, 'median': 145.71946828596018}


 10%|█         | 10/100 [01:12<08:21,  5.57s/it][20:20:12] Can't kekulize mol.  Unkekulized atoms: 4 5 6 28 50
[20:20:27] Can't kekulize mol.  Unkekulized atoms: 4 5 6 60 67
 20%|██        | 20/100 [03:27<11:03,  8.29s/it][20:23:21] SMILES Parse Error: extra open parentheses while parsing: c1ccc2c(c1)sc1c3c(c(N)c2c1O)sc1c4c(c(N)c3c12)sc2c1c(c(N)c3c24)sc1c4c(c(N)c5c6ccccc6sc3c54)sc3c4c(c(N)c6c7ccccc7sc1c36)sc1c3c(c(N)c5c67ccccc67sc3c15)sc1c3c(c(N)c4c5ccccc5sc3c12)sc1c2c(c(N)c3c45ccccc45sc2c31)sc1c2c(c(N)c3c4ccccc4sc2c13)sc1c2c(c(N)c3c4ccccc4sc2c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2c(c(N)c3c21)sc1c2

{'min': 109.35727087851915, 'max': 233.04558899989934, 'avg': 157.83248152285935, 'median': 148.60558380074718}


  5%|▌         | 5/100 [00:15<03:50,  2.43s/it][20:45:45] SMILES Parse Error: syntax error while parsing: c1ccc(Nc2cc(-c3ccc(Nc4cc(-c5ccc(Nc6cc(-c7ccc(Nc8cc(-c9ccc(Nc%10cc(-c%11ccc(Nc%12cc(-c%13ccc(N)cc%13)cc%12)cc%11)cc%10)cc%9)cc%8)cc%7)cc%6)cc%5)cc%4)cc%3)cc%2)cc%1)cc2cc(-c3ccccc3)cc(N)c12
[20:45:45] SMILES Parse Error: check for mistakes around position 123:
[20:45:45] %12)cc%11)cc%10)cc%9)cc%8)cc%7)cc%6)cc%5)
[20:45:45] ~~~~~~~~~~~~~~~~~~~~^
[20:45:45] SMILES Parse Error: Failed parsing SMILES 'c1ccc(Nc2cc(-c3ccc(Nc4cc(-c5ccc(Nc6cc(-c7ccc(Nc8cc(-c9ccc(Nc%10cc(-c%11ccc(Nc%12cc(-c%13ccc(N)cc%13)cc%12)cc%11)cc%10)cc%9)cc%8)cc%7)cc%6)cc%5)cc%4)cc%3)cc%2)cc%1)cc2cc(-c3ccccc3)cc(N)c12' for input: 'c1ccc(Nc2cc(-c3ccc(Nc4cc(-c5ccc(Nc6cc(-c7ccc(Nc8cc(-c9ccc(Nc%10cc(-c%11ccc(Nc%12cc(-c%13ccc(N)cc%13)cc%12)cc%11)cc%10)cc%9)cc%8)cc%7)cc%6)cc%5)cc%4)cc%3)cc%2)cc%1)cc2cc(-c3ccccc3)cc(N)c12'
 31%|███       | 31/100 [03:23<10:35,  9.22s/it][20:48:42] Can't kekulize mol.  Unkekulized atoms: 0 1 2 

(['Nc1cc(N)c(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)c(O)c1O',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)c(O)c(N)c1O',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4ccc(N)cc4)c(-c4ccc(N)cc4)c3)c(-c3ccc(N)cc3)c2)cc(-c2ccc(N)cc2)c1O',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4ccccc4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)cc(-c2ccccc2)c1O.Cc1cc(N)c(C)c(C)c1O',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)c(O)c(N)c1',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)cc(-c2ccccc2)c1',
  'Nc1cc(N)c(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)cc1-c1ccccc1',
  'Nc1cc(N)c(-c2cc(N)c(-c3cc(N)c(-c4ccc(N)cc4)c(-c4ccc(N)cc4)c3)c(-c3ccc(N)cc3)c2)c(-c2ccc(N)cc2)c1O',
  'Nc1cc(-c2cc(N)c(-c3cc(N)c(-c4cc(N)c(-c5ccccc5)c(-c5ccccc5)c4)c(-c4ccccc4)c3)c(-c3ccccc3)c2)cc(-c2ccccc2)c1O',
  'Nc1cc(-c2cc(-c3cc(-c4cc(-

In [62]:
new_mols_df = pd.read_csv('gen_algo_cache/population.csv')
new_mols_df = new_mols_df[~new_mols_df.SMILES.isin(initial_molecules)]

In [63]:
new_mols_df['SMILES'].head(20).to_csv('submit.csv', encoding='UTF-8', index=False)

In [70]:
import os
import zipfile
with zipfile.ZipFile('submission.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('submit.csv', os.path.basename('submit.csv'))
    zipf.write('4_ml_PDSC_prediction.ipynb', os.path.basename('4_ml_PDSC_prediction.ipynb'))
    zipf.write('5_rl_genetic_algo.ipynb', os.path.basename('5_rl_genetic_algo.ipynb'))
    zipf.write('src/utils.py', os.path.basename('src/utils.py'))
    zipf.write('src/llm_interface.py', os.path.basename('src/llm_interface.py'))